In [3]:
from dotenv import load_dotenv
from llama_index.llms.openrouter import OpenRouter
from llama_index.core.llms import ChatMessage
from os import getenv

load_dotenv()

/home/tphung/projects/lawrag/env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [4]:
llm = OpenRouter(
    max_tokens=256,
    context_window=4096,
    model="cohere/command-r"
)

In [12]:
message = ChatMessage(role="user", content="Tell me a joke")
resp = llm.chat([message])
print(resp)

assistant: What do you call a factory that makes distinctly average products?

*An adequate-ory!*


In [5]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader



In [14]:
documents = SimpleDirectoryReader("poems_data").load_data()

# Trying out Cohere embeddings

In [6]:
from llama_index.embeddings.cohere import CohereEmbedding

embed_model = CohereEmbedding(
    api_key=getenv("COHERE_API_KEY"),
    model_name="embed-multilingual-light-v3.0",
    input_type="search_query",
)

In [7]:
from llama_index.llms.cohere import Cohere

llm_original = Cohere(model="command-r", api_key=getenv("COHERE_API_KEY"))

In [17]:
llm.chat([ChatMessage(role="user", content="Rancontrez-moi une blague")])

ChatResponse(message=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content='Une blague pour vous !\n\nDeux atomes hydrogenes se réunissent dans un bar. Le premier hydrogène dit à l\'autre: "j\'ai perdu mon electron."\n\nL\'autre demande: "Vraiment? Comment ça s\'est passé?"\n\nLe premier répond: "Je ne suis pas sûr, ça s\'est passé tellement vite!"\n\nEst-ce que cela vous a fait rire?', additional_kwargs={}), raw=ChatCompletion(id='gen-Rh4ZBrrpzaaDSE7El5mbHEujVQ0E', choices=[Choice(finish_reason='COMPLETE', index=0, logprobs=None, message=ChatCompletionMessage(content='Une blague pour vous !\n\nDeux atomes hydrogenes se réunissent dans un bar. Le premier hydrogène dit à l\'autre: "j\'ai perdu mon electron."\n\nL\'autre demande: "Vraiment? Comment ça s\'est passé?"\n\nLe premier répond: "Je ne suis pas sûr, ça s\'est passé tellement vite!"\n\nEst-ce que cela vous a fait rire?', role='assistant', function_call=None, tool_calls=None))], created=1722938026, model='cohere/command-r

In [8]:
from llama_index.core import Settings

Settings.llm=llm
Settings.embed_model=embed_model

In [19]:
index = VectorStoreIndex.from_documents(
    documents=documents,
)

In [9]:
from llama_index.postprocessor.cohere_rerank import CohereRerank

cohere_rerank = CohereRerank()

In [21]:
query_engine = index.as_query_engine(
    postprocessors=[cohere_rerank],
)

In [22]:
response = query_engine.query("A qui semble le poete?")

In [23]:
response

Response(response='Le Poète semble au prince des nuées.', source_nodes=[NodeWithScore(node=TextNode(id_='1ac37012-aaaf-479f-af51-b7e962afdcd6', embedding=None, metadata={'file_path': '/home/tphung/projects/lawrag/poems_data/albatros.txt', 'file_name': 'albatros.txt', 'file_type': 'text/plain', 'file_size': 762, 'creation_date': '2024-08-05', 'last_modified_date': '2024-08-05'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='f7eda0f4-8215-4846-9f7a-013d728bdd03', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'file_path': '/home/tphung/projects/lawrag/poems_data/albatros.txt', 'file_name': 'albatros.txt', 'file_type': 'text/plain', 'file_size': 762, 'creation_date': '2024-08-05', 'last_modified_date': '202

# example of Vectore storage persisting

In [24]:
index.storage_context.persist(persist_dir="./poems_storage") # save the index to ./storage directory

In [25]:
from llama_index.core import load_index_from_storage, StorageContext
from llama_index.core.storage.docstore import SimpleDocumentStore
from llama_index.core.storage.index_store import SimpleIndexStore
from llama_index.core.vector_stores import SimpleVectorStore

storage_context = StorageContext.from_defaults(
    persist_dir="poems_storage"
    # theese arguments do not work but they were in guide
    # docstore=SimpleDocumentStore.from_persist_dir(persist_dir="storage"),
    # index_store=SimpleIndexStore.from_persist_dir(persist_dir="storage"),

    # vector_store=SimpleVectorStore.from_persist_dir(persist_dir="storage"),
)
persistent_index = load_index_from_storage(storage_context=storage_context)

In [26]:
persistent_engine = persistent_index.as_query_engine(
    postprocessors=[cohere_rerank],
)

In [27]:
response_new = persistent_engine.query("A qui semble le poete?")
response_new

Response(response='Le Poète semble au prince des nuées.', source_nodes=[NodeWithScore(node=TextNode(id_='1ac37012-aaaf-479f-af51-b7e962afdcd6', embedding=None, metadata={'file_path': '/home/tphung/projects/lawrag/poems_data/albatros.txt', 'file_name': 'albatros.txt', 'file_type': 'text/plain', 'file_size': 762, 'creation_date': '2024-08-05', 'last_modified_date': '2024-08-05'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='f7eda0f4-8215-4846-9f7a-013d728bdd03', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'file_path': '/home/tphung/projects/lawrag/poems_data/albatros.txt', 'file_name': 'albatros.txt', 'file_type': 'text/plain', 'file_size': 762, 'creation_date': '2024-08-05', 'last_modified_date': '202

# CSV file as data source

In [57]:
from llama_index.readers.file import PandasCSVReader, PagedCSVReader, CSVReader
from pathlib import Path

# CSVReader does not work properly, so i use PandasCSVReader
reader = PandasCSVReader()
file = Path("code_du_travail.csv")
travail_documents = reader.load_data(file=file)


article_id, source_id, article_num, path_title, texte, embedding, order_num, etat, ref_textes, date_deb, date_fin, update_date, versions
LEGIARTI000017961623, LEGITEXT000006072050, L1, ["Partie législative", "Chapitre préliminaire : Dialogue social."], <p>Tout projet de réforme envisagé par le Gouvernement qui porte sur les relations individuelles et collectives du travail, l'emploi et la formation professionnelle et qui relève du champ de la négociation nationale et interprofessionnelle fait l'objet d'une concertation préalable avec les organisations syndicales de salariés et d'employeurs représentatives au niveau national et interprofessionnel en vue de l'ouverture éventuelle d'une telle négociation. </p><p>A cet effet, le Gouvernement leur communique un document d'orientation présentant des éléments de diagnostic, les objectifs poursuivis et les principales options. </p><p>Lorsqu'elles font connaître leur intention d'engager une telle négociation, les organisations indiquent égaleme

In [74]:
travail_index =VectorStoreIndex.from_documents(
    documents=travail_documents,
)

TooManyRequestsError: status_code: 429, body: data=None message="You are using a Trial key, which is limited to 100 API calls / minute. You can continue to use the Trial key for free or upgrade to a Production key with higher rate limits at 'https://dashboard.cohere.com/api-keys'. Contact us on 'https://discord.gg/XW44jPfYJu' or email us at support@cohere.com with any questions"

In [44]:
travail_query_engine = travail_index.as_query_engine(
    postprocessors=[cohere_rerank],
)

In [45]:
resp = travail_query_engine.query("Quel est le but du travail?")

In [46]:
for key, value in resp.__dict__.items():
    print(key, value)

response Les domaines dans lesquels les procédures de concertation et de négociation du gouvernement sont mises en œuvre sont ceux des relations individuelles et collectives du travail, de l'emploi et de la formation professionnelle.
source_nodes [NodeWithScore(node=TextNode(id_='c25c21a7-a34c-4045-9e55-8084ce2ac366', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='5f0c1caf-6e15-40d0-82a1-8f0b2118ef5c', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='80a15fe069f9c8b63b7f6250328fe8dbe8ede6b0f564d8612b361f50eab32560')}, text='article_id: LEGIARTI000017961625\nsource_id: LEGITEXT000006072050\narticle_num: L2\npath_title: ["Partie législative", "Chapitre préliminaire : Dialogue social."]\ntexte: <p>Le Gouvernement soumet les projets de textes législatifs et réglementaires élaborés dans le champ défini par l\'article <a href=\'/affichCodeArticle.do?cidTexte=LEGITEXT0

In [59]:
resp.source_nodes[0].get_text().split("\n")

['article_id: LEGIARTI000017961625',
 'source_id: LEGITEXT000006072050',
 'article_num: L2',
 'path_title: ["Partie législative", "Chapitre préliminaire : Dialogue social."]',
 "texte: <p>Le Gouvernement soumet les projets de textes législatifs et réglementaires élaborés dans le champ défini par l'article <a href='/affichCodeArticle.do?cidTexte=LEGITEXT000006072050&idArticle=LEGIARTI000017961623&dateTexte=&categorieLien=cid' title='Code du travail - art. L1 (V)'>L. 1</a>, au vu des résultats de la procédure de concertation et de négociation, à la Commission nationale de la négociation collective, de l'emploi et de la formation professionnelle dans les conditions prévues à l'article <a href='/affichCodeArticle.do?cidTexte=LEGITEXT000006072050&idArticle=LEGIARTI000006901822&dateTexte=&categorieLien=cid'>L. 2271-1</a>.</p>",
 'embedding: ',
 'order_num: 1610612735',
 'etat: VIGUEUR',
 'ref_textes: ',
 'date_deb: 1622505600000',
 'date_fin: 32472144000000',
 'update_date: 2023-12-15',
 've

In [60]:
resp_deux = travail_query_engine.query("Donnex moi texte d article LEGIARTI000017961625")

In [65]:

resp_deux.get_formatted_sources()

'> Source (Doc id: ccf7732e-7b7c-4f99-b12d-3b07f4aa281e): article_id: LEGIARTI000006900781\nsource_id: LEGITEXT000006072050\narticle_num: L1111-1\npath_title:...\n\n> Source (Doc id: c25c21a7-a34c-4045-9e55-8084ce2ac366): article_id: LEGIARTI000017961625\nsource_id: LEGITEXT000006072050\narticle_num: L2\npath_title: ["Pa...'

In [63]:
resp_deux.source_nodes[0].get_text().split("\n")

['article_id: LEGIARTI000006900781',
 'source_id: LEGITEXT000006072050',
 'article_num: L1111-1',
 'path_title: ["Partie législative", "Première partie : Les relations individuelles de travail", "Livre Ier : Dispositions préliminaires", "Titre Ier : Champ d\'application et calcul des seuils d\'effectifs", "Chapitre unique."]',
 "texte: <p>Les dispositions du présent livre sont applicables aux employeurs de droit privé ainsi qu'à leurs salariés.</p><p>Elles sont également applicables au personnel des personnes publiques employé dans les conditions du droit privé, sous réserve des dispositions particulières ayant le même objet résultant du statut qui régit ce personnel.</p>",
 'embedding: ',
 'order_num: 42949',
 'etat: VIGUEUR',
 'ref_textes: ',
 'date_deb: 1209600000000',
 'date_fin: 32472144000000',
 'update_date: 2023-12-15',
 'versions:']

# Elastic search as db

In [23]:
from llama_index.vector_stores.elasticsearch import ElasticsearchStore, AsyncDenseVectorStrategy

es = ElasticsearchStore(
    index_name="my_index",
    es_url="http://localhost:9200",
    retrieval_strategy=AsyncDenseVectorStrategy(),
)

In [32]:
from llama_index.core.schema import TextNode

movies = [
    TextNode(
        text="The lives of two mob hitmen, a boxer, a gangster and his wife, and a pair of diner bandits intertwine in four tales of violence and redemption.",
        metadata={"title": "Pulp Fiction"},
    ),
    TextNode(
        text="When the menace known as the Joker wreaks havoc and chaos on the people of Gotham, Batman must accept one of the greatest psychological and physical tests of his ability to fight injustice.",
        metadata={"title": "The Dark Knight"},
    ),
    TextNode(
        text="An insomniac office worker and a devil-may-care soapmaker form an underground fight club that evolves into something much, much more.",
        metadata={"title": "Fight Club"},
    ),
    TextNode(
        text="A thief who steals corporate secrets through the use of dream-sharing technology is given the inverse task of planting an idea into thed of a C.E.O.",
        metadata={"title": "Inception"},
    ),
    TextNode(
        text="A computer hacker learns from mysterious rebels about the true nature of his reality and his role in the war against its controllers.",
        metadata={"title": "The Matrix"},
    ),
    TextNode(
        text="Two detectives, a rookie and a veteran, hunt a serial killer who uses the seven deadly sins as his motives.",
        metadata={"title": "Se7en"},
    ),
    TextNode(
        text="A family heads to an isolated hotel for the winter where a sinister presence influences the father into violence, while his psychic son sees horrific forebodings from both past and future.",
        metadata={"title": "The Shining"},
    ),
    
]

In [12]:
type(travail_documents[0])

llama_index.core.schema.Document

In [ ]:
storage_context = StorageContext.from_defaults(
    
)

In [54]:
from llama_index.core import StorageContext, VectorStoreIndex

def print_results(results):
    for rank, result in enumerate(results, 1):
        print(
            f"{rank}. title={result.metadata['title']} score={result.get_score()} text={result.get_text()}"
        )


def search(
    vector_store: ElasticsearchStore, nodes: list[TextNode], query: str
):
    storage_context = StorageContext.from_defaults(vector_store=vector_store)
    index = VectorStoreIndex(nodes, storage_context=storage_context)
    index.insert_nodes()

    print(">>> Documents:")
    retriever = index.as_retriever()
    results = retriever.retrieve(query)
    print(results)

    print("\n>>> Answer:")
    query_engine = index.as_query_engine()
    response = query_engine.query(query)
    print(response)

In [51]:
chunk = travail_documents[:100]
search(es, [], "quel est le but d article LEGIARTI000006900832")

>>> Documents:
[NodeWithScore(node=TextNode(id_='d0025684-268d-4f48-9f53-2ff25b083650', embedding=None, metadata={'filename': 'code_du_travail.csv', 'extension': '.csv'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='LEGIARTI000033437828, LEGITEXT000006072050, L1134-7, ["Partie législative", "Première partie : Les relations individuelles de travail", "Livre Ier : Dispositions préliminaires", "Titre III : Discriminations", "Chapitre IV : Actions en justice.", "Section 2 : Dispositions spécifiques à l\'action de groupe"], <p align=\'left\'>Une organisation syndicale de salariés représentative au sens des articles <a href=\'/affichCodeArticle.do?cidTexte=LEGITEXT000006072050&idArticle=LEGIARTI000006901582&dateTexte=&categorieLien=cid\' title=\'Code du travail - art. L2122-1 (V)\'>L. 2122-1</a>, <a href=\'/affichCodeArticle.do?cidTexte=LEGITEXT000006072050&idArticle=LEGIARTI000019347598&dateTexte=&categorieLien=cid\' title=\'Code du travail - art. 

In [61]:

chunk = travail_documents[:100]
search(es, chunk, "quel est le but d article L1134-5")

ConnectionError: Connection error caused by: ConnectionError(Connection error caused by: ClientConnectorError(Cannot connect to host localhost:9200 ssl:default [Connect call failed ('127.0.0.1', 9200)]))

In [66]:
es.is_embedding_query
storage_context = StorageContext.from_defaults(vector_store=es)

In [67]:

index = VectorStoreIndex(travail_documents[:20], storage_context=storage_context)

In [69]:
len(travail_documents

1

In [68]:
from time import sleep

storage_context = StorageContext.from_defaults(vector_store=es)
for i in range(21, len(travail_documents), 20):
    chunk = travail_documents[i:i + 20]
    out = index.insert_nodes(chunk)
    print(i, out)
    sleep(4)